In [1]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [3]:
import machine

In [3]:
# Set uart port and tx rx
uart = machine.UART(1)

# Set part baud and data bits, Parity bits, stop bits

uart.init(19200, tx = 26, rx = 0)

In [116]:
print(uart.readline())

b'2707\n'


In [4]:
uart_out_buf = bytearray(1)
uart.write("@OF30")
uart_out_buf = 0x0a
uart.write(b'\x0a')
uart.write("@RG2")
uart_out_buf = 0x0a
uart.write(b'\x0a')

In [6]:
uart.write("jak tam na swiecie \nwszystko ok?\n")

In [5]:


def internet_set_time():
    try:    
        import usocket as socket
    except:
        import socket
    try:
        import ustruct as struct
    except:
        import struct

    # (date(2000, 1, 1) - date(1900, 1, 1)).days * 24*60*60
    NTP_DELTA = 3155673600

    # The NTP host can be configured at runtime by doing: ntptime.host = 'myhost.org'
    host = "pool.ntp.org"

    print("setting up time")
    def time():
        NTP_QUERY = bytearray(48)
        NTP_QUERY[0] = 0x1B
        addr = socket.getaddrinfo(host, 123)[0][-1]
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            s.settimeout(1)
            res = s.sendto(NTP_QUERY, addr)
            msg = s.recv(48)
        finally:
            s.close()
        val = struct.unpack("!I", msg[40:44])[0]
        return val - NTP_DELTA


    # There's currently no timezone support in MicroPython, and the RTC is set in UTC time.
    
    
    def settime():
        t = time()
        import machine
        import utime

        tm = utime.gmtime(t)
        machine.RTC().datetime((tm[0], tm[1], tm[2], tm[6] + 1, tm[3], tm[4], tm[5], 0))
        print(machine.RTC().datetime())
    settime()
    
    
def connect_internet_set_time(network_name = "igor_test",password = "mywifikey"):
    import network
    from network import WLAN
    wlan = WLAN(network.STA_IF)
    wlan.active(True)   
    nets = wlan.scan()


    if not wlan.isconnected():
        print(nets)

        for net in nets:

            if net[0].decode() == network_name:
                print('Network found!')
                wlan.connect(net[0].decode(),  password)
                while not wlan.isconnected():
                    machine.idle() # save power while waiting
                print('WLAN connection succeeded!')


                break
    try:
        internet_set_time()
    except:
        "could not set time"

import time        
        
def get_time_ns():
    return time.time_ns()

In [6]:
connect_internet_set_time()

[(b'MAINS-2G', b'\x00\x0fa\x82\xbe`', 11, -63, 4, False), (b'MAINS-2G', b'\x10`K6]0', 1, -78, 4, False), (b'35A697D758A02016382D65C34AE5088A', b'\x00\x1ds@\x8fa', 11, -85, 2, False), (b'MAINS-2G', b'\x00\x0fa\x82\xbb\x80', 1, -87, 4, False), (b'', b'\x96\x9aJ@S#', 5, -93, 3, False), (b'TP-Link_2G', b'\x90\x9aJ@S#', 5, -93, 3, False)]
setting up time


In [7]:
print(get_time_ns())

247960327000


  Serial2.begin(19200, SERIAL_8N1, 0, 26);         // RX=0   TX=26
    Serial2.print("@OF30");           // Offset
    Serial2.write(0x0a);
    Serial2.print("@RG2");            // MAG=2
    Serial2.write(0x0a);
    fConv = (float)LCDHEIGHT/(float)MAXADRANGE; 

In [8]:
import ulab
from ulab import numpy as np


In [134]:
heart_rate_vector = np.array((0 for x in range(150)), dtype=np.uint8)


In [ ]:
n_mes = 0
while True:
    result = uart.readline()

    if result:
        try:
            number = result.decode()

            if number[0] == "#": #special case
                print("n messages",n_mes)
                n_mes = 0
                if number[1] == "-":
                    #pass
                    print("error")
                else:
                    try :
                        Rri_n = int(number[1:].strip())
                        
                    except Exception as e:
                        print(e)
                        #print(number[1],"whole string",number)
                    #Rri = number[1:].strip()
                    print(Rri_n)
            else:
                n_mes +=1
        except Exception as e:
            print(e)


n messages 211
452
n messages 229
error
.n messages 521
error
n messages 260
string index out of range
n messages 251
error
n messages 260
error
n messages 261
error
n messages 260
error
.n messages 261
error
n messages 901
error
n messages 260
984
.n messages 256
980
n messages 261
1016
n messages 254
984
n messages 260
980
n messages 260
996
n messages 264
1016
n messages 253
984
.n messages 258
1028
n messages 273
1068
n messages 256
1012
n messages 264
1032
n messages 261
980
n messages 244
968
.n messages 253
996
n messages 273
1048
n messages 260
984
n messages 271
1048
n messages 253
1080
n messages 99
error
.n messages 1042
712
n messages 421
error
.n messages 469
944
n messages 1045
error
.n messages 597
error
n messages 410
632
.n messages 1253
460
..
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

aes 

In [54]:

print(machine.RTC().datetime())


import time
print(time.time_ns())

(2022, 4, 25, 0, 7, 19, 20, 584867)
704186360593276000


In [76]:
from bluetooth_simple.ble_simple_peripheral import *
ble = bluetooth.BLE()
p = BLESimplePeripheral(ble)

Starting advertising


In [78]:
p.send("hello")

[leftinbuffer] ['New connection 0']


In [114]:
### import bluetooth
from bluetooth_simple.ble_simple_peripheral import *

import struct
def make_heartrate_frame(heartrate):
    precise_time = get_time_ns()
    packed = struct.pack("II",precise_time,heartrate)
    return packed


def main_program():
    ble = bluetooth.BLE()
    uart_b = BLESimplePeripheral(ble)
    try:
        connect_internet_set_time()
    except:
        pass
    old_time = get_time_ns() 
    while True:
        result = uart.readline()

        if result:
            try:
                number = result.decode()

                if number[0] == "#": #special case
                    if number[1] == "-":
                        pass
                        #print("error")
                    else:
                        heartrate = 60000.0 / float(number[1:])

                        #lcd.clear(lcd.BLACK)

                        #lcd.text(lcd.CENTER, lcd.CENTER, str(heartrate))

                        #print("heart rate",heartrate)
                else: #just the heartbyte
                #
                    #print(number.strip())
                    
                    if uart_b.is_connected():
                        #p.send("hello")
                        try:
                            new_time = get_time_ns()
                            precise_time = get_time_ns()
                            
                            
                            uart_b.send(struct.pack("II",precise_time,int(number[1:])))
                            #p.send("hello")
                            print(new_time - old_time)
                            old_time = new_time
                        except Exception as e:
                            print(uart_b._connections)
                            print(e)
                            conn = list(uart_b._connections)[0]  # hopefully one connection
                            try:
                                uart_b._ble.gap_disconnect(conn)
                                uart_b._connections.clear()
                            except Exception as e2:
                                print(e2)
                            
                    else:
                        pass
                        #print("not connected")
                    
            except UnicodeError:
                print("UnicodeError")
                
main_program()

Starting advertising
New connection 0
{0}
invalid syntax for integer with base 10
Disconnected 0
Traceback (most recent call last):
  File "bluetooth_simple/ble_simple_peripheral.py", line 57, in _irq
KeyError: 0
...........

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 68, in <module>
  File "<stdin>", line 24, in main_program
KeyboardInterrupt: 


In [ ]:
/*----------------------------------------------
 *  M5StickC Plus Pulse Serial Recieve Sample
 *    #2019/06/06 - #2019/06/08 - #2020/07/22
 *    Serial 19200ボー
 *    No Speaker
 *    
 *    Board Select -> M5Stick-C
  -----------------------------------------------*/
#include <M5StickCPlus.h>
#include <WiFi.h>

#define MAXADRANGE  4096
#define DELIMITCODE  0x0a       // Delimit Code 

#define SWEEPMAG    8 
#define LCDWIDTH    240      // M5Stick-Plus  240
#define LCDHEIGHT   135      // M5Stick-Plus  135
#define PLSPOSX     32
#define PLSPOSY     2

static  int sweepptr = 0;
static  int lastY = 0;
static  int plsrate = 0;
static  int x = 0;
static  float fConv;  

void setup() {
    M5.begin();
    pinMode(36, INPUT);
    gpio_pulldown_dis(GPIO_NUM_25);
    gpio_pullup_dis(GPIO_NUM_25);    
//---
    M5.Axp.ScreenBreath(8);         // LCD輝度を抑える 7-15
    M5.Lcd.setRotation(1);          // 回転をDefaultに戻す
    M5.Lcd.fillScreen(BLACK);
//--- 省電力でWifiOFFする場合に下記入れる
    WiFi.mode(WIFI_OFF);
//--
    M5.Lcd.setTextColor(GREEN);
    M5.Lcd.setTextSize(3);    
//--
    delay(500);
    Serial2.begin(19200, SERIAL_8N1, 0, 26);         // RX=0   TX=26
    Serial2.print("@OF30");           // Offset
    Serial2.write(0x0a);
    Serial2.print("@RG2");            // MAG=2
    Serial2.write(0x0a);
    fConv = (float)LCDHEIGHT/(float)MAXADRANGE;  
    WiFi.mode(WIFI_OFF);
}

void loop() {
String   strInput;  
int      val,y;

 if(Serial2.available() > 0) {
    strInput = Serial2.readStringUntil(DELIMITCODE);
    if (strInput[0]=='#'){    
   // 脈拍数データ識別
      strInput[0]=' ';        // @をSpaceに置き換える
      if (strInput[1]=='-'){
        strInput[1]=' ';          // マイナスはErrorマークなのでSpaceに置き換えて色を赤にする
        M5.Lcd.setTextColor(RED);     // エラー時は赤文字
      }else{
        M5.Lcd.setTextColor(GREEN);   // 正常脈拍時は緑文字
      }
      val = strInput.toInt();     
      if (val>0) plsrate = 60000/val;                     // Interval(ms)を1分間の脈拍数(bpm)に変換
      M5.Lcd.fillRect(PLSPOSX, PLSPOSY, 50, 16, BLACK);   //　脈拍数表示領域だけ背景を抜く
    }else{                
  // 脈波形データ処理
      sweepptr++;
      if (sweepptr>=SWEEPMAG){
        sweepptr = 0;
        val = strInput.toInt();       
        M5.Lcd.drawLine(x, 0, x, LCDHEIGHT-1, BLACK);
        y = LCDHEIGHT - (int)((float)val*fConv);
        if (x > 0) {
          M5.Lcd.drawLine(x - 1, lastY, x, y, WHITE);
        }else{
          M5.Lcd.drawLine(x, lastY, x, y, WHITE);
        }
        lastY = y;
        x++;
        if (x >= LCDWIDTH) x=0;
      }
    }
  //　脈拍数を波形の上に上書き    
    M5.Lcd.setCursor(PLSPOSX, PLSPOSY);
    M5.Lcd.println(plsrate);      
  }
}